In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/home/baldeeb/Code/pytorch-NOCS')

from models.nocs import get_nocs_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_Weights
from habitat_datagen_util.utils.dataset import HabitatDataset
from torch.utils.data import DataLoader
from habitat_datagen_util.utils.collate_tools import collate_fn
from torch.optim import Adam
import torch

DATA_DIR = "/home/baldeeb/Code/pytorch-NOCS/data/habitat/train"  # larger dataset
device='cuda:1'

model = get_nocs_resnet50_fpn(
                maskrcnn_weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT,
                nocs_num_bins=32,
                nocs_loss_mode = 'classification',
                multiheaded_nocs = True,

                )
model.to(device).train()
habitatdata = HabitatDataset(DATA_DIR)
dataloader = DataLoader(habitatdata, 
                        batch_size=2, 
                        shuffle=True, 
                        collate_fn=collate_fn)
optimizer = Adam(model.parameters())

In [2]:
# Helpers
def targets2device(targets, device):
    for i in range(len(targets)): 
        for k in ['masks', 'labels', 'boxes']: 
            targets[i][k] = targets[i][k].to(device)
    return targets

In [3]:
model.roi_heads.training_mode('multiview')
for itr, (images, targets) in enumerate(dataloader):
    images = images.to(device)
    targets = targets2device(targets, device)
    losses = model(images, targets)
    loss = sum(losses.values())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

/home/baldeeb/miniconda3/envs/torch-nocs/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB (GPU 0; 23.66 GiB total capacity; 20.43 GiB already allocated; 923.00 MiB free; 21.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF